# Parallel comuting

In this example, we demonstrate how parallel computing - a method of performing multiple calculations simultaneously to improve efficiency - is applied to estimate the probability of levee failure due to wave overtopping. 

First, we import the necessary packages:

In [6]:
from probabilistic_library import ReliabilityProject, DistributionType, ReliabilityMethod
import time

We consider the limit state function for wave overtopping (which we have artificially slowed down):

In [7]:
from utils.models import ZFunctionOvertopping

And the following reliability project:

In [8]:
def define_project():

    project = ReliabilityProject()
    project.model = ZFunctionOvertopping.z_sleep
    
    project.variables["h"].distribution = DistributionType.log_normal
    project.variables["h"].mean = 1.5
    project.variables["h"].deviation = 0.05

    project.variables["hm0"].distribution = DistributionType.log_normal
    project.variables["hm0"].mean = 1.5
    project.variables["hm0"].deviation = 0.25

    project.variables["tm10"].distribution = DistributionType.log_normal
    project.variables["tm10"].mean = 3
    project.variables["tm10"].deviation = 0.5

    project.variables["wave_direction"].distribution = DistributionType.deterministic
    project.variables["wave_direction"].mean = 0.0

    project.variables["dike_normal"].distribution = DistributionType.deterministic
    project.variables["dike_normal"].mean = 0.0

    project.variables["y_crest"].distribution = DistributionType.deterministic
    project.variables["y_crest"].mean = 6.0

    project.variables["q_crit"].distribution = DistributionType.log_normal
    project.variables["q_crit"].mean = 0.001
    project.variables["q_crit"].deviation = 0.01

    project.settings.reliability_method = ReliabilityMethod.crude_monte_carlo
    project.settings.minimum_samples = 1000
    project.settings.maximum_samples = 1000
    project.settings.variation_coefficient = 0.02

    return project

We perform the reliability calculations using the `crude_monte_carlo` method:

In [9]:
project = define_project()

project.run()

from utils.printing import print_results
print_results(project.design_point, project.settings)

Beta = 1.2265281200366105
Probability of failure = 0.10999999999999993
self: alpha = 0.0, x = 0.0
h: alpha = -0.04342108406821088, x = 1.5018303649169757
hm0: alpha = -0.37159130122483325, x = 1.595531805516477
tm10: alpha = -0.6051390499098042, x = 3.3460139874125105
wave_direction: alpha = 0.0, x = 0.0
dike_normal: alpha = 0.0, x = 0.0
y_crest: alpha = 0.0, x = 6.0
q_crit: alpha = 0.702738389862569, x = 1.5619634827850045e-05
Not converged (convergence = 0.08994948077064754 > 0.02)
Model runs = 1001


If not specified, the calculations are performed on a single processor. To utilize multiple processors, we adjust the setting `project.settings.max_parallel_processes`. With the following code, we analyze the effect of using multiple processors on the calculation time:

In [10]:
max_parallel_processes = [1, 4, 8]

for val in max_parallel_processes:

    project = define_project()
    project.settings.max_parallel_processes = val

    t = time.time()
    project.run()
    elapsed = time.time() - t

    print(f"Max parallel processes: {val}")
    print(f"Reliability index: {project.design_point.reliability_index}")
    print(f"Time elapsed: {elapsed} seconds")
    print(f"----------------------------------------------------------------")

Max parallel processes: 1
Reliability index: 1.2265281200366105
Time elapsed: 152.16503143310547 seconds
----------------------------------------------------------------
Max parallel processes: 4
Reliability index: 1.2265281200366105
Time elapsed: 26.041754722595215 seconds
----------------------------------------------------------------
Max parallel processes: 8
Reliability index: 1.2265281200366105
Time elapsed: 13.702514410018921 seconds
----------------------------------------------------------------
